# From Graphs to Probability Distributions: A Hands-On Introduction to ERGMs
#### By Tom Talpir (tom.talpir@weizmann.ac.il)



### Introduction

Graphs are extremely useful objects for describing many complex systems. From molecules and neural networks in the brain to social networks, graphs provide a natural way to represent interactions between components and capture both local and global structure. Understanding the structural properties of these graphs, and the design principles that govern their connectivity, is therefore crucial for understanding the phenomena they describe.

For example, suppose we are given the connectome of a worm and are interested in studying the rules and patterns underlying its neuronal connectivity. One approach would be to perform a detailed analysis of this specific observed graph in isolation, but such an analysis would inevitably be biased toward this particular sample. Instead, we can take a statistical approach and build a model that assigns a probability to every possible graph as a function of a set of structural features. Such a model allows us to ask which structural features are truly important for explaining the observed connectivity. Intuitively, we would expect structural properties that play a key role in shaping the connectivity to be associated with stronger model parameters, while irrelevant properties should have little or no effect.

Exponential Random Graph Models (**ERGMs**) are a class of statistical models designed for exactly this purpose. An ERGM defines a probability distribution over graphs, where the likelihood of a graph depends on a set of chosen structural features (such as the number of reciprocal connections), weighted by model parameters that quantify their importance.


### Tutorial goals

This tutorial has two main learning goals. We will fit an ERGM to observed network data by finding the optimal model parameters. In doing so, we will introduce and use two important statistical tools:

* Maximum Likelihood Estimation (**MLE**) via **gradient descent**
* Markov Chain Monte Carlo (**MCMC**) simulations

We will demonstrate these ideas using two example graphs:
* **Sampson’s monastery network** — a small graph consisting of 18 nodes, used as an initial toy example. Each node represents a monk in a New England monastery, and a directed edge indicates the presence of a friendly relationship between two monks.
* **C. elegans connectome** — a graph consisting of 279 nodes, representing the sensory neurons, interneurons, and motor neurons in the nervous system of *C. elegans*.

### Step 1 - What is a graph and an ERGM?

Formally, a graph $G$ consists of $n$ nodes that are connected to one another by a set of edges. The connectivity is represented by a binary connectivity matrix $W$, where $W_{i,j} = 1$ if nodes $i$ and $j$ are connected, and $0$ otherwise. The graph can be **directed**, meaning that edges have a direction, or **undirected**, in which case $W_{i,j} = W_{j,i}$.

An ERGM defines a random variable $\mathbf{Y}$, which represents a random graph on $n$ nodes. The probability of observing a specific graph $y$ is given by
$$
\Pr(\mathbf{Y}=y \mid \theta) = \frac{\exp(\theta^T g(y))}{\sum_{z \in \mathcal{Y}} \exp(\theta^T g(z))},
$$
where $\mathcal{Y}$ is the set of all graphs on $n$ nodes, $g(y)$ is a vector of graph statistics describing $y$, and $\theta \in \mathbb{R}^q$ is a vector of model parameters.

In essence, an ERGM defines a probability distribution over all possible graphs on $n$ nodes.

<div style="text-align: center;">
  <img src="./Haber_ERGM_illustration.png" width="300">
  <p style="font-size:12px;"><em>(Haber et al., 2023)</em></p>
</div>

### Step 2 - Getting to know the data

We begin by loading the Sampson monastery graph, represented as an $18 \times 18$ binary connectivity matrix. $W_{i,j}$ indicates that monk $i$ claimed to have a friendly relationship with monk $j$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1234)

W = np.array(
    [[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
     [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
     [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
     [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
     [1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
     [1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
     [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
     [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
     [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
     [1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
     [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
     [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
     [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]]
)

n_nodes = W.shape[0]

We will use 2 structural features for our model -
* Number of edges - how many relationships exist in the graph
* Number of reciprocal edges - the number of reciprocal relationships, where $W_{i,j} = W_{j, i}$

**[TASK]** In the next cell, complete the `get_number_of_edges` function.

In [ ]:
def get_number_of_edges(W: np.ndarray):
    """
    Counts the number of edges in a directed graph

    Parameters:
         W: An n by n binary connectivity matrix

    Returns:
        Number of edges in the graph
    """
    number_of_edges = None

    #####################
    ### COMPLETE THIS ###
    #####################

    return number_of_edges


def get_number_of_reciprocal_edges(W: np.ndarray):
    """
    Counts the number of reciprocal edges in a directed graph. Here we use a neat trick of calculating W^2 to find nodes that have a path of length 2

    Parameters:
         W: An n by n binary connectivity matrix

    Returns:
        Number of reciprocal edges in the graph
    """
    return (W * W.T).sum() / 2

def calculate_graph_statistics(W: np.ndarray, feature_calculators):
    """
    Given a directed graph W and a list of callable feature calculators, we calculate the graph statistics.

    Parameters:
        W: Connectivity matrix for which the statistics should be calculated
        feature_calculators: A list of callable feature calculators. Each feature calculator should be a callable object that receives a connectivity matrix, and returns the feature statistics.

    Returns:
        A numpy array containing the graph statistics.
    """
    num_features = len(feature_calculators)
    statistics = np.zeros(num_features)

    for i, feature_calculator in enumerate(feature_calculators):
        statistics[i] = feature_calculator(W)

    return statistics


Let's calculate the graph statistics using the 2 features we just defined.
<br>
**[TASK]** Verify that the graph has **88 edges** and **28 reciprocal edges**.

In [ ]:
feature_calculators = [get_number_of_edges, get_number_of_reciprocal_edges]
observed_stats = calculate_graph_statistics(W, feature_calculators)
print(f"Number of edges - {observed_stats[0]}")
print(f"Number of reciprocal edges - {observed_stats[1]}")

### Step 3 - Fitting the model

A common approach for estimating the parameters of a probability distribution is to maximize a likelihood function, such that under the assumed statistical model, the observed data is the most probable outcome. This procedure is known as Maximum Likelihood Estimation (**MLE**).

The likelihood function assigns a probability to the observed data under different choices of model parameters. In our case, the likelihood can be written as
$$
\mathcal{L}(\theta \mid y_{\text{obs}}) = \frac{\exp(\theta^T g(y_{\text{obs}}))}{\sum_{z \in \mathcal{Y}} \exp(\theta^T g(z))},
$$
where $y_{\text{obs}}$ denotes the observed graph.

In this toy example, we define two structural features: the number of edges and the number of reciprocal edges. Accordingly, $g(y)$ is a vector of two statistics.


It is common practice to work with the log-likelihood function instead, as it is more convenient to work with -
$$
\ell (\theta \mid y_{\text{obs}}) = \theta^T g(y_{\text{obs}}) - \log\!\left(\sum_{z \in \mathcal{Y}} \exp(\theta^T g(z))\right)
$$

We now wish to find the model parameters that maximize the log-likelihood function, denoted by $\theta^*$. This is equivalent to minimizing the negative log-likelihood, which is often more convenient since many optimization algorithms are formulated as minimization problems:
$$
\theta^* = \arg \max \ell (\theta \mid y_{\text{obs}}) = \arg \min \big(-\ell (\theta \mid y_{\text{obs}})\big)
$$

We will start with the simplest optimization algorithm: **gradient descent**. Gradient descent is an iterative method for minimizing a differentiable function. At each step, the parameters are updated by moving in the opposite direction of the gradient, with the goal of converging to a minimum of the function. Formally, the update rule is given by
$$
\theta_{i+1} = \theta_i - \eta \nabla \ell(\theta),
$$
where $\eta$ is the learning rate.


We now need to compute the gradient of the log-likelihood function. It is perfectly fine to skip the derivation on a first read, but the underlying idea is relatively simple and, with a bit of patience, should become clear.

The log-likelihood function can be differentiated with respect to $\theta$ to obtain the gradient -
$$
\nabla \ell(\theta) = \frac{\partial}{\partial \theta} \ \ell (\theta) = g(y_{\text{obs}}) - \frac{\sum_{z\in\mathcal{Y}} \exp(\theta^Tg(z))g(z)}{\sum_{z\in\mathcal{Y}} \exp(\theta^Tg(z))}
$$

$$
= g(y_{\text{obs}}) - \sum_{z\in\mathcal{Y}} \frac{\exp(\theta^Tg(z))}{Z}g(z)
$$

$$
= g(y_{\text{obs}})- \sum_{z\in\mathcal{Y}}\Pr_{\theta, \mathcal{Y}}(\mathbf{Y}=z)g(z)
$$

$$
= g(y_{\text{obs}})- \mathbb{E}_{z\sim\mathcal{Y}}[g(z)]
$$

where $Z=\sum_{z\in\mathcal{Y}} \exp(\theta^Tg(z))$ is the normalization factor.

Notice that the gradient decomposes into two terms - the statistics of the observed graph, $g(y_{\text{obs}})$, which we can compute directly, and the expected statistics over all possible graphs, $\mathbb{E}_{z \sim \mathcal{Y}}[g(z)]$. Computing this expectation exactly is computationally infeasible, as it would require iterating over all directed graphs on $n$ nodes. Even for a relatively small graph such as the Sampson network, this corresponds to $2^{n^2 - n} = 2^{306}$ possible graphs.

This is precisely where Markov Chain Monte Carlo (**MCMC**) simulations become useful, as we will see in the next step.


### Step 4 – MCMC simulations

As we have just seen, we cannot directly compute $\mathbb{E}_{z \sim \mathcal{Y}}[g(z)]$ due to the exponential number of possible graphs that would need to be considered. An alternative is to approximate this expectation by computing the sample mean of $g(z)$ over a collection of sampled graphs. This naturally raises the question - how should we choose this sample?

One naive approach would be to generate graphs by independently flipping a coin with probability $p$ for each possible edge. However, this approach is too simplistic, and it is unlikely that such samples would faithfully represent the target distribution. We therefore need a more principled method, which is where Markov Chain Monte Carlo (**MCMC**) techniques come into play.

MCMC refers to a class of algorithms used to draw samples from probability distributions. In this tutorial, we will focus on the Metropolis–Hastings algorithm, which is a widely used MCMC method.

In its simplest form, the Metropolis–Hastings algorithm generates samples by iteratively performing the following steps -

1. Start from an initial graph $y_0 \in \mathcal{Y}$ and set it as the current graph, $y_{\text{current}} = y_0$.
2. Propose a new graph $y_{\text{proposed}}$ by making a small modification to $y_{\text{current}}$ by adding or removing an edge between a randomly chosen pair of nodes $i, j$.
3. Compute the *change score*, defined as $\delta_g(y)_{i,j} = g(y_{\text{proposed}}) - g(y_{\text{current}})$.
4. Accept the proposed graph with probability $p_{\text{accept}} = \min\left(1, \exp\!\left(\theta^T \delta_g(y)_{i,j}\right)\right)$.

Steps 2–4 are repeated for a large number of iterations, until a sufficient number of graphs have been collected.

Once we have obtained a sample of graphs, we can approximate the expectation $\mathbb{E}_{z \sim \mathcal{Y}}[g(z)]$ using the sample mean.

**[TASK]** Complete the following MCMC sampling function

In [ ]:
def MCMC_graph_sample(thetas: np.ndarray,
                      feature_calculators,
                      n_nodes: int,
                      num_graphs=1000,
                      seed_graph=None,
                      seed_graph_edge_probability=0.1,
                      burn_in=1000,
                      steps_per_sample=10
                      ):
    """
    Run an MCMC sampling process sample graphs from the current distribution.

    Parameters:
        thetas: current  distribution parameters, of size `# of features`.
        feature_calculators: a list of callable feature calculators.
        n_nodes: number of nodes in the graph.
        num_graphs: number of graphs to sample.
        seed_graph_edge_probability: Edge probability of the seed graph, generated from an Erdős–Rényi model.
        burn_in: number of graphs that aren't collected to the final sample, as part of the burn in process.
        steps_per_sample: number of edge flips before evaluating a candidate graph.

    Returns:
        A collection of graphs, represented as numpy arrays, sampled from the distribution.
    """
    if seed_graph is None:
        seed_graph = np.random.choice([0, 1], size=(n_nodes, n_nodes), p=[1-seed_graph_edge_probability, seed_graph_edge_probability])
        seed_graph[np.diag_indices(n_nodes)] = 0

    sampled_graphs = []

    current_graph = seed_graph
    current_statistics = calculate_graph_statistics(current_graph, feature_calculators)

    while len(sampled_graphs) < num_graphs:
        #####################
        ### COMPLETE THIS ###
        #####################
        continue

    return sampled_graphs

def calculate_mean_statistics(sample_graphs: list, feature_calculators):
    """
    Calculate the mean statistics over a collection of graphs.

    Parameters:
        sample_graphs: a list of graphs, each graph is a numpy array.
        feature_calculators: a list of callable feature calculators.

    Returns:
        The mean statistics over the provided graphs, of size `# of features`.
    """
    statistics = np.zeros((len(sample_graphs), len(feature_calculators)))

    for i, W in enumerate(sample_graphs):
        statistics[i] = calculate_graph_statistics(W, feature_calculators)

    return np.mean(statistics, axis=0)


### Step 5 – Running the optimization loop

We have now reached the final step - training the model. We start from the observed graph and an initial set of model parameters, and then iteratively apply the gradient descent update rule.

**[TASK]** In the next cell, complete the gradient computation based on the derivation in the previous steps.


In [ ]:
feature_calculators = [get_number_of_edges, get_number_of_reciprocal_edges]
observed_stats = calculate_graph_statistics(W, feature_calculators)
num_features = len(observed_stats)

initial_thetas = np.random.uniform(-1, 1, num_features)
num_opt_steps = 100
learning_rate = 0.001
MCMC_number_of_graphs_per_sample = 100
MCMC_burn_in = 0
MCMC_seed_graph_density = np.sum(W) / (n_nodes * (n_nodes-1))

seed_graph = np.random.choice([0, 1], size=(n_nodes, n_nodes), p=[1-MCMC_seed_graph_density, MCMC_seed_graph_density])
seed_graph[np.diag_indices(n_nodes)] = 0

print(f"Observed statistics - {observed_stats}")
print(f"Seed MCMC graph density - {MCMC_seed_graph_density}")
print(f"Initial thetas: {initial_thetas}")
gradients = []
thetas = initial_thetas

all_graphs = []
all_thetas = []
for i in range(num_opt_steps):
    all_thetas.append(thetas)
    print(f"Working on step {i+1}/{num_opt_steps}")

    if i == 0:
        burn_in = MCMC_burn_in
    else:
        burn_in = 0

    sample_graphs = MCMC_graph_sample(
        thetas=thetas,
        feature_calculators=feature_calculators,
        n_nodes=n_nodes,
        num_graphs=MCMC_number_of_graphs_per_sample,
        seed_graph=seed_graph,
        burn_in=burn_in,
        steps_per_sample=5
    )

    print(f"\tSampled {len(sample_graphs)} graphs")

    sample_statistics = calculate_mean_statistics(sample_graphs, feature_calculators)
    print(f"\tMean statistics: {sample_statistics}")

    gradient = None
    #####################
    ### COMPLETE THIS ###
    #####################

    gradients.append(gradient)
    print(f"\tGradient: {gradient}")

    thetas = thetas - learning_rate * gradient

    # Use the last sampled graph for the next starting point of the MCMC chain
    seed_graph = sample_graphs[-1]

    all_graphs.extend(sample_graphs)

### Step 6 – Evaluating the model

After training the model, we want to evaluate its performance. Run the following cell, which should generate three figures -

* Gradients over training iterations  
* Number of edges across MCMC-sampled graphs  
* Model parameters over training iterations  

**[TASK]** Analyze each figure and try to answer the following questions - 

* How would you expect the gradient to behave as the optimization progresses?
* Explain the behavior of the second figure (number of edges across the MCMC-sampled graphs). Why does it oscillate? Around what value is it oscillating? Is there anything special about that value?
* Are the model parameters stable? Do they appear to converge to a specific value?

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 3), dpi=200)
# fig, ax = plt.subplots(nrows=1, ncols=3)

ax[0].plot(np.mean(gradients, axis=1))
ax[0].set_title("Gradient over time")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Gradient")

ax[1].plot([np.sum(g) for g in all_graphs])
ax[1].set_title("# edges across MCMC graphs")
ax[1].set_xlabel("Number of graphs")
ax[1].set_ylabel("# edges")

ax[2].plot(all_thetas)
ax[2].set_title("Model parameters")
ax[2].set_xlabel("Epoch")
ax[2].set_ylabel("Theta")

plt.tight_layout()
plt.show()

### Step 7 – Train an ERGM model for the worm connectome

Our final step is to repeat everything we have learned on a more interesting network - the *C. elegans* connectome.

**[TASK]** Recreate the training loop from the previous steps, but this time for the worm connectome. This is a much larger network, so do not expect good results on the first attempt. You will likely need to tune hyperparameters such as the MCMC settings, the learning rate, and the number of optimization iterations.


In [ ]:
worm_path = "./worm.npy"

W = np.load(worm_path)
n_nodes = W.shape[0]

### Step 8 – Future tutorial ideas and directions

* **Improved MCMC sampling** – MCMC offers more advanced techniques that can substantially improve sampling efficiency and lead to better mixing.
* **Improved optimization** – Our current optimization approach is very basic, relying on vanilla gradient descent. Possible extensions include:
  * More advanced algorithms such as Newton–Raphson
  * Better initialization of the parameters $\theta$ (production-level ERGM implementations often use an approach known as MPLE, which includes a neat trick based on logistic regression)
* **Convergence criteria** – In the current implementation, training stops after a fixed number of iterations. A more robust approach would be to continue training until convergence is detected. Identifying convergence is a nontrivial problem and may require the use of more sophisticated statistical tests.